In [1]:
!pip install fastai==2.4
import os
import glob
import time
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, lab2rgb

import torch
from torch import nn, optim
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from fastai.vision.learner import create_body
from torchvision.models.resnet import resnet18
from fastai.vision.models.unet import DynamicUnet

  Using cached fastai-2.4-py3-none-any.whl (187 kB)
  Using cached fastcore-1.3.29-py3-none-any.whl (55 kB)
INFO: pip is looking at multiple versions of fastai to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch<1.10,>=1.7.0 (from fastai) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch<1.10,>=1.7.0


Creating models

In [10]:
class ClassifierHead(nn.Module):
  def __init__(self, input_size=512*8*8, output_size=5):
        super().__init__()

        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(input_size, 1000)
        self.fc2 = nn.Linear(1000, output_size)

  def forward(self, x):
      x = self.fc1(x)
      return self.fc2(self.relu(x))

class ColorizationHead(nn.Module):
  def __init__(self, input_c=1, output_c=2, n_down=5, num_filters=512):
        super().__init__()
        self.model_layers = []
        self.model_layers += (self.get_layer(num_filters, num_filters))
        for _ in range(n_down - 5):
            self.model_layers += (self.get_layer(num_filters, num_filters, dropout=True))
        out_filters = num_filters
        for _ in range(3):
            self.model_layers += (self.get_layer(out_filters // 2, out_filters))
            out_filters //= 2
        self.model_layers += (self.get_layer(output_c, out_filters, outermost=True))
        self.model = nn.Sequential(*self.model_layers)

  def forward(self, x):
      print("A")
      return self.model(x)

  def get_layer(self, nf, ni, bias=False, dropout=False, outermost=False):
        layer = []
        conv = nn.ConvTranspose2d(ni, nf, kernel_size=4,
                                              stride=2, padding=1, bias=bias)
        relu = nn.ReLU(False)

        layer.append(relu)
        layer.append(conv)


        if outermost:
          layer.append(nn.Tanh())
        else:
          layer.append(nn.BatchNorm2d(nf))

        if dropout:
          layer.append(nn.Dropout(0.5))

        return layer

class MLTModel(nn.Module):
  def __init__(self, head="multitask"):
          super().__init__()
          self.resnet_body = create_body(resnet18(), pretrained=True, n_in=n_input, cut=-2)
          self.heads = []
          if head=="multitask":
            self.classifier_head = ClassifierHead()
            self.colorization_head = ColorizationHead()
          elif head=="classifier":
            pass
          else:
            pass
  def forward(self, x):
      x = self.resnet_body(x)
      out_classifier = self.classifier_head(x.view(-1, 512*8*8))
      out_colorization_head = self.colorization_head(x)
      return out_classifier, out_colorization_head

In [11]:
n_input = 1

x = torch.zeros(1,1, 256, 256)

# body = create_body(resnet18(), pretrained=True, n_in=n_input, cut=-2)
# y = body(x)
# print(y.shape)
# x = x.view(-1, 512*8*8)

model = MLTModel()
out_classifier, out_colorization_head = model(x)
print(out_classifier.shape)
print(out_colorization_head.shape)

A
torch.Size([1, 5])
torch.Size([1, 2, 256, 256])
